In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-dotenv openAI langchain_core langchain_openai

In [ ]:
# 환경변수 설정

In [6]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI


In [14]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/drive/MyDrive/Colab Notebooks/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

In [16]:
llm.invoke("안녕")

'하세요.\n\n서울대학교에 재학 중인 학생입니다. 제 이름은 홍길동이며, 행정학과에 소속되어 있습니다. 학교 생활을 하면서 다양한 경험과 지식을 쌓고 있습니다.\n\n제가 행정학과를 선택한 이유는 정부나 공공기관에서 일하는 것에 관심이 있기 때문입니다. 또한 이 분야에서는 사회 문제를 해결하는 데에 기여할 수 있는 역할을 할 수 있다는 점이 매력적으로 다가왔습니다. 특히 현재 우리 사회에서는 다양한 문제들이 발생하고 있는데, 이를 해결하기 위해 정부와 시민들이 함께 협력해야 한다고 생각합니다.\n\n서울대학교에서는 행정학과의 다양한 분야를'

In [13]:
# 예제 1-1 <오픈AI API를 사용하여 주제에 대한 간단한 설명 생성 파이프라인 구축>

# 라이브러리 불러오기
import openai
from typing import List

# 기본 오픈AI 클라이언트 사용
client = openai.OpenAI()

# "Hello!" 메시지를 보내고 응답을 받음
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "안녕하세요!"}]
)
response.choices[0].message.content

# 설명에 사용할 프롬프트 템플릿 정의
prompt_template = "주제 {topic}에 대해 짧은 설명을 해주세요"

# 메시지를 보내고 모델의 응답을 받는 함수
def call_chat_model(messages: List[dict]) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    return response.choices[0].message.content

# 주어진 주제에 따라 설명을 요청하는 함수
def invoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

# "더블딥" 주제로 설명 요청
invoke_chain("더블딥")

'더블딥(Double Dip)은 일반적으로 두 번의 혜택을 동시에 받는 것을 의미합니다. 이 용어는 주로 경제나 금융 분야에서 사용되며, 한 가지 상황에서 두 가지 혜택을 동시에 누리는 경우를 가리킵니다. 이는 일반적으로 공정하지 않은 혜택의 이중 획득을 의미하며, 비윤리적인 행동으로 인정받을 수 있습니다. 그러나 일부 상황에서는 정당한 이중 혜택을 받을 수도 있습니다.'

In [21]:
# 예제 1-2 <랭체인을 사용하여 주제에 대한 간단한 설명 생성 파이프라인 구축>

# 라이브러리 불러오기
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
# 미스트랄AI 모델을 사용할 경우 주석 해제
# from langchain_mistralai.chat_models import ChatMistralAI

# 주어진 주제에 대해 짧은 설명을 요청하는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template(
    "주제 {topic}에 대해 짧은 설명을 해주세요"
)

# 출력 파서를 문자열로 설정
output_parser = StrOutputParser()
# 오픈AI의 gpt-3.5-turbo 모델을 사용하여 채팅 모델 설정
model = ChatOpenAI(model="gpt-3.5-turbo")
# 미스트랄AI 모델을 사용할 경우 주석 해제
#model = ChatMistralAI()

# 파이프라인 설정: 주제를 받아 프롬프트를 생성하고, 모델을 통해 응답을 생성한 후 문자열로 파싱
chain = (
    {"topic": RunnablePassthrough()}  # 입력 받은 주제를 그대로 통과시킴
    | prompt                         # 프롬프트 템플릿을 적용
    | model                          # 모델을 사용해 응답 생성
    | output_parser                  # 응답을 문자열로 파싱
)

# "더블딥" 주제로 설명 요청
chain.invoke("더블딥")

'더블딥(Double Dip)은 일반적으로 두 번의 혜택을 동시에 받는 것을 의미합니다. 이 용어는 주로 경제 분야에서 사용되며, 예를 들어 하나의 행위로 두 가지 이윤을 얻는 경우를 가리킬 때 사용됩니다. 더블딥은 일반적으로 긍정적인 의미로 사용되지만, 때로는 부정적인 뜻으로도 사용될 수 있습니다.'